[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/geo-di-lab/emerge-lessons/blob/main/docs/activities/intro_globe_analysis.ipynb)

# EMERGE Activity: Data Analysis & Mapping of GLOBE Data

This lesson shows how to investigate GLOBE mosquito & land cover data, calculate statistics, and create charts & maps at global, state, and county levels.

## Introduction to the Tools
*   **Python** (a popular programming knowledge for analyzing data, automating tasks, creating software, training machine learning models, and more) to analyze GLOBE Mosquito Habitat Mapper and Land Cover data.
*   **Google Colab** (a free, cloud-based coding environment) to run Python on the browser. *No setup or downloads necessary!*
*   **Google Earth Engine** (a cloud-based platform that includes satellite imagery and other geospatial datasets) to load environmental data.

**To run the code:**
> Make sure you have this notebook open in Google Colab. If you are starting from the digital textbook, click [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/geo-di-lab/emerge-lessons/blob/main/docs/activities/intro_globe_analysis.ipynb)

Each block of code is called a cell. To run a cell, hover over it and click the arrow in the top left of the cell, or click inside of the cell and press `Shift + Enter`.

Note: When you run a block of code for the first time, Google Colab will say `Warning: This notebook was not authored by Google`. Please click `Run Anyway`.

In [ ]:
# Import libraries
import pandas as pd                           # For working with data
pd.set_option("display.max_columns", None)    # Lets us see all columns of the data instead of just a preview
import geopandas as gpd                       # For working with spatial data
import numpy as np                            # For working with numbers
from datetime import date                     # For working with dates
import matplotlib.pyplot as plt               # For making graphs
from matplotlib.colors import to_rgb          # For getting colors
import branca.colormap as cm                  # For creating color scales
import seaborn as sns                         # For more options to load colors and create plots
import folium                                 # For creating interactive maps

In the code above, we imported Python libraries, which expand the options we have with our code. Each library comes with many functions designed to accomplish specific tasks, like loading a dataset, performing calculations, making a chart, and more.

In [ ]:
# This is a comment (added using # at the start), used by programmers to explain their code
# Comments do not impact how the code runs

# Print today's date
today = date.today()
print(f"Today's date is {today}.")

A great feature of Google Colab is that you are able to write Python code and see the output directly on your browser!

## Mosquitoes Around the GLOBE

Let's load the data directly from the link (everything stays in Google Colab on the browser; nothing gets downloaded to your computer)! This is a processed version of the data that has the columns renamed and some errors corrected.

The data comes from GLOBE Observer accessed through [the API](https://www.globe.gov/globe-data/globe-api). You can see all the steps taken to process and clean the data in [chapter 1 of the EMERGE curriculum](https://geo-di-lab.github.io/emerge-lessons/docs/ch1/lesson6.html).

In [ ]:
mosquito = gpd.read_file('https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/globe_mosquito.zip')
mosquito.head()

See a list of the columns:

In [ ]:
mosquito.info()

How many rows are in the dataset?

In [ ]:
len(mosquito)

There were 43,012 citizen science contributions from 2018 to 2024. Now, let’s see the number of countries where people submitted data.

In [ ]:
len(mosquito['CountryCode'].unique())

Let's see the types of the habitats (water sources) the citizen scientists recorded.

In [ ]:
# General water source types
mosquito['WaterSourceType'].value_counts()

These are the general types of water sources that citizen scientists reported to NASA GLOBE. It looks like most of the data collected were about artificial containers. Let's see some of the more specific types in the other column:

In [ ]:
# More specific water source types
mosquito['WaterSource'].value_counts()

Let's make a pie chart using the more general types, WaterSourceType

In [ ]:
# Here are some options for color palettes
display(sns.color_palette(palette='rainbow'))
display(sns.color_palette(palette='CMRmap'))
display(sns.color_palette(palette='BrBG'))
display(sns.color_palette(palette='cubehelix'))
display(sns.color_palette(palette='Set2'))
display(sns.color_palette(palette='Set3'))
display(sns.color_palette(palette='tab20'))

In [ ]:
# Pie chart of water types
types = mosquito[['SiteId', 'WaterSourceType']].groupby('WaterSourceType', as_index=False).count()

# Create pie chart
plt.figure(figsize=(5, 5))
patches, texts = plt.pie(colors = sns.color_palette('Set2'), # Enter the name of the color palette you want to use
                         x = types['SiteId'])
plt.title("GLOBE Mosquito Sightings: Water Source Types (General)")
plt.legend(patches, types['WaterSourceType'],
           loc = 'center left', bbox_to_anchor=(1, 0.5), frameon=False)
plt.show()

What is the average larvae count by country?

In [ ]:
mosquito_avg = mosquito.groupby('CountryCode')['LarvaeCountProcessed'].mean()
mosquito_avg

Let's make a map showing the larvae count by country. The [country boundaries (generalized)](https://hub.arcgis.com/datasets/esri::world-countries-generalized/about) are from Esri, Garmin, and U.S. Central Intelligence Agency (The World Factbook). The boundaries are generalized to allow data processing and visualizations to load faster. The ISO alpha-3 codes come from the [World Countries](https://hub.arcgis.com/datasets/esri::world-countries/about) layer from Esri, Garmin, U.S. Central Intelligence Agency (The World Factbook), and International Organization for Standardization (ISO).

In [ ]:
# Load country boundaries
countries = gpd.read_file('https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/world_countries_general.geojson').to_crs(epsg=4326)

# Add mosquito data for each country
mosquito_avg = countries.merge(mosquito_avg, left_on='iso3', right_on='CountryCode', how='left')

In [ ]:
fig, ax = plt.subplots(figsize = (10, 4))

# Create plot of average larvae count by country
mosquito_avg.plot(column = 'LarvaeCountProcessed', cmap = 'viridis',
                     legend = True, vmin = 0, vmax = 50, ax = ax,
                     missing_kwds = {'color': 'lightgrey'})
plt.title('GLOBE Mosquito Sightings: Average Larvae Count')
ax.axis('off')
plt.show()

Now, we'll make an *interactive* map showing total GLOBE observations by country.

In [ ]:
# Get total GLOBE observations by country
mosquito_obs = mosquito.groupby('CountryCode').size() \
                       .reset_index(name='GLOBE_Observations')
mosquito_obs = countries.merge(mosquito_obs, left_on='iso3', right_on='CountryCode', how='left').dropna(subset=['GLOBE_Observations'])

In [ ]:
# Load a color scale, starting at 1 and ending at 100
# Blue indicates 100 or more GLOBE observations, while green/yellow indicate closer to 1 observation
colors = cm.linear.YlGnBu_03.scale(1, 100)
colors

In [ ]:
map = folium.Map(location=[0, 0], zoom_start=3, tiles="CartoDB positron")

# Create interactive map of GLOBE observations by country
folium.GeoJson(
    geo_data = mosquito_obs.to_json(),
    data = mosquito_obs,
    key_on = "feature.properties.name",
    tooltip = folium.features.GeoJsonTooltip(
        fields = ['name', 'GLOBE_Observations'],
        aliases = ['Country:', 'Observations:']
    ),
    style_function = lambda feature: {
        "fillColor": colors(feature['properties']['GLOBE_Observations']),
        'fillOpacity': 0.8,
        'color': 'grey',
        'weight': 1
    }
).add_to(map)

display(map)

## Land Cover Around the GLOBE

Like the mosquito data, we'll load in the GLOBE land cover data directly from the link.

In [ ]:
land_cover = gpd.read_file('https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/globe_land_cover.zip')
land_cover.head()

In [ ]:
len(land_cover)

A helpful part of the land cover dataset is the MUC classifications. MUC (Modified UNESCO Classification) is a classification system with different land use types that helps us understand habitats around the world.

What are the most common MUC codes by country?

In [ ]:
# Find the most common MUC for each country
muc = land_cover.groupby('CountryCode')['MucDescription'] \
    .apply(lambda x: x.value_counts().idxmax() if not x.value_counts().empty else None).reset_index(name='MucDescription')

# Add a column for the number of GLOBE observations with the MUC code
muc['Count'] = land_cover.groupby('CountryCode')['MucDescription'] \
    .apply(lambda x: x.value_counts().max()).values

# Add a column for the total number of GLOBE observations
muc['GLOBE_Observations'] = land_cover.groupby('CountryCode').size().values

muc

In [ ]:
# Add this data to the country boundaries
muc = countries.merge(muc, left_on='iso3', right_on='CountryCode', how='left')

# Create general categories
muc_list = ['Barren', 'Closed Forest', 'Cultivated', 'Herbaceous', 'Open Water', 'Trees', 'Urban', 'Wetlands', 'Woodland']

# Simplify some names into the categories listed above
for muc_code in muc_list:
    muc.loc[muc['MucDescription'].str.contains(muc_code, na=False), 'MucDescriptionShort'] = muc_code

In [ ]:
fig, ax = plt.subplots(figsize = (11, 5))

# Create a map of the most common MUC codes by country
muc.plot(column = 'MucDescriptionShort', cmap = 'viridis',
                     legend = True, ax = ax,
                     missing_kwds = {'color': 'lightgrey', 'label': 'No Data'},
                     legend_kwds={'loc': 'lower left', 'frameon': False})
plt.title('GLOBE Land Cover: Most Common MUC Codes')
plt.show()

Notice the gray areas. There were no GLOBE observations recorded for these countries, so we'll remove these from the dataset to make it easier to visualize the data.

In [ ]:
muc = muc.dropna(subset=['GLOBE_Observations'])

### Challenge: Create an interactive map for land cover
Like the mosquito observation map above, let's make a map of the number of GLOBE Observations for land cover. In addition to the name of the country and the number of GLOBE Observations, add the most common MUC code (`MucDescriptionShort`) to the text box that pops up when you hover over each country.

Use the following columns:
- `name` for the name of the country (pop-up)
- `MucDescriptionShort` for the most common MUC code (pop-up)
- `GLOBE_Observations` for the number of GLOBE observations recorded (pop-up and color of each country)

In summary, each country's color on the map will be based on `GLOBE_Observations` and the pop-up should include the country name, number of GLOBE observations, and the most common MUC.

Feel free to reference the code for making an interactive map of the mosquito observations (which we've pasted below). If you get stuck, click `Open for Answer` to see one way to create this map!

```
map = folium.Map(location=[0, 0], zoom_start=3, tiles="CartoDB positron")

# Create interactive map of GLOBE observations by country
folium.GeoJson(
    geo_data = mosquito_obs.to_json(),
    data = mosquito_obs,
    key_on = "feature.properties.name",
    tooltip = folium.features.GeoJsonTooltip(
        fields = ['name', 'GLOBE_Observations'],
        aliases = ['Country:', 'Observations:']
    ),
    style_function = lambda feature: {
        "fillColor": colors(feature['properties']['GLOBE_Observations']),
        'fillOpacity': 0.8,
        'color': 'grey',
        'weight': 1
    }
).add_to(map)

display(map)
```

In [ ]:
map = folium.Map(location=[0, 0], zoom_start=3, tiles="CartoDB positron")

# Replace ? with your code below
folium.GeoJson(
    geo_data = ?,
    data = ?,
    key_on = "feature.properties.name",
    tooltip = folium.features.GeoJsonTooltip(
        fields = ?,
        aliases = ?
    ),
# Replace ? with your code above
    style_function = lambda feature: {
        "fillColor": colors(feature['properties']['GLOBE_Observations']),
        'fillOpacity': 0.8,
        'color': 'grey',
        'weight': 1
    }
).add_to(map)

display(map)

In [ ]:
#@title Open for Answer

map = folium.Map(location=[0, 0], zoom_start=3, tiles="CartoDB positron")

# Replace ? with your code below
folium.GeoJson(
    geo_data = muc.to_json(),
    data = muc,
    key_on = "feature.properties.name",
    tooltip = folium.features.GeoJsonTooltip(
        fields = ['name', 'GLOBE_Observations', 'MucDescriptionShort'],
        aliases = ['Country:', 'Observations:', 'Most common MUC:']
    ),
# Replace ? with your code above
    style_function = lambda feature: {
        "fillColor": colors(feature['properties']['GLOBE_Observations']),
        'fillOpacity': 0.8,
        'color': 'grey',
        'weight': 1
    }
).add_to(map)

display(map)

## Mosquitoes in Your State

Now, we'll view the GLOBE data for your state and later on, your county. Enter the name of your state below.

The boundary files from the [U.S. Census](https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html). We will use the same mosquito data as the first part of this code notebook (which comes from GLOBE Observer accessed through [the API](https://www.globe.gov/globe-data/globe-api) from [chapter 1 of our digital textbook](https://geo-di-lab.github.io/emerge-lessons/docs/ch1/lesson6.html)).

In [ ]:
state_name = "Your State Name"

# For example,
# state_name = "Florida"

In [ ]:
# Load county boundaries
counties = gpd.read_file('https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/us_counties.zip').to_crs('EPSG:4326')

# Filter to all counties in state
state_counties = counties.loc[counties['STATE_NAME'] == state_name]

# Get state boundary by combining the counties together
state = state_counties[['geometry']].dissolve()

state.plot()

In [ ]:
# Create empty map zoomed to the state
map = folium.Map(tiles="Cartodb dark_matter")

# Get all GLOBE mosquito data within the state
mosquito_state = gpd.sjoin(mosquito, state, how="inner", predicate='intersects') \
          .drop(columns=['index_right']) \
          .reset_index(drop=True)

# Add county boundaries to the map
folium.GeoJson(
    state_counties.to_json(),
    name='County Boundaries',
    tooltip=folium.features.GeoJsonTooltip(fields=['NAMELSAD'], aliases=['County:']),
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'grey',
        'weight': 1
    }
).add_to(map)

# Add each point as a green circle on the map
for idx, row in mosquito_state.iterrows():
    popup_content = f"<b>Date:</b> {row['MeasuredDate']}<br><b>Water Source:</b> {row['WaterSourceType']}<br><b>Longitude:</b> {row['MeasurementLongitude']}<br><b>Latitude:</b> {row['MeasurementLatitude']}"
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        popup=folium.Popup(popup_content, max_width=300),
        radius=6,
        color='black',
        weight=1,
        fillColor='lightgreen',
        fillOpacity=0.5
    ).add_to(map)

# Add an option to hide the county boundaries
folium.LayerControl().add_to(map)

# Zoom to the state
minx, miny, maxx, maxy = state.bounds.values[0]
bounds = [[miny, minx], [maxy, maxx]]
map.fit_bounds(bounds)

# Display the map
display(map)


See if there are any points in your county! If not, find a nearby county that has points. Enter the name of your chosen county below:

In [ ]:
county_name = "Your County Name"

# Make sure it ends in the word "County"

# For example,
# county_name = "Broward County"

## Mosquitoes in Your County

Get the specific outline of your county:

In [ ]:
county = state_counties.loc[counties['NAMELSAD'] == county_name]
county.plot()

Now, we use those boundaries to filter for all GLOBE points within your county, from 2018 to 2024.

In [ ]:
data_county = mosquito.sjoin(county, how="inner", predicate="within")

num_total = len(data_county)

print(f"There were {num_total} GLOBE points recorded within {county_name} from 2018-2024 by community scientists.")

num_eliminated = len(data_county[data_county['BreedingGroundEliminated'] == 'true'])
print(f"Of those points, {num_eliminated} ({round(num_eliminated * 100 / num_total)}%) were successfully mitigated by the community scientists, which reduces the risk for mosquitoes inhabiting that location in the future.")

If there were no GLOBE points in your county, please choose a nearby county that has at least one point. You can check by looking at the map of your state above.

In [ ]:
data_county.head()

How many mosquito observations have been submitted to GLOBE Observer each year in your county? We'll make a bar plot to figure this out.

In [ ]:
# Add a new column for year
data_county['MeasuredYear'] = data_county['MeasuredAt'].dt.year

# Make histogram of mosquito sightings each year
years = data_county[['SiteId', 'MeasuredYear']].groupby('MeasuredYear', as_index=False).count()
plt.bar(years['MeasuredYear'], years['SiteId'])
plt.title("Mosquito Sightings by Year", loc = 'left')
plt.title(county_name, loc = 'right')
plt.show()

Let's make a pie chart of the water source types (both general and specific) where mosquitoes were reported in this county.

In [ ]:
# Get counts of each water source (general)
types = data_county[['SiteId', 'WaterSourceType']].groupby('WaterSourceType', as_index=False).count()

# Create pie chart
plt.figure(figsize=(5, 5))
patches, texts = plt.pie(x = types['SiteId'],
                         colors = sns.color_palette('Set2'))
plt.title(f"GLOBE Mosquito Sightings in {county_name}: Water Source Types (General)")
plt.legend(patches, types['WaterSourceType'],
           loc = 'center left', bbox_to_anchor=(1, 0.5), frameon=False)
plt.show()

In [ ]:
# Get counts of each water source (specific)
types = data_county[['SiteId', 'WaterSource']].groupby('WaterSource', as_index=False).count()

# Create pie chart
plt.figure(figsize=(5, 5))
patches, texts = plt.pie(x = types['SiteId'],
                         colors = sns.color_palette('Set2'))
plt.title(f"GLOBE Mosquito Sightings in {county_name}: Water Source Types (Specific)")
plt.legend(patches, types['WaterSource'],
           loc = 'center left', bbox_to_anchor=(1, 0.5), frameon=False)
plt.show()

## Remote Sensing Data for Your County

If you have not used Google Earth Engine before, follow [this tutorial](https://geo-di-lab.github.io/emerge-lessons/docs/ch1/lesson5a.html) to create an Earth Engine account and project.

In [ ]:
# Write your project ID for Google Earth Engine below
projectID = "Write Your Project ID"

# For example,
# projectID = "emerge-lessons-23148"

In [ ]:
import ee
import geemap
import geopandas as gpd
ee.Authenticate()

# Write your project ID here, in quotes
ee.Initialize(project = projectID)

In [ ]:
# Convert county boundaries to Earth Engine format
region = geemap.geopandas_to_ee(county)

# Create an empty map zoomed to the county
county_center = county.iloc[0].geometry.centroid

map = geemap.Map(center=[county_center.y, county_center.x],
                 zoom=10)
map.add_basemap('CartoDB.DarkMatter')

Now, we will map land surface temperature across the county. First, define a date range which would be used to find the average temperature across those dates.

In [ ]:
start_date = "2025-06-01"
end_date = "2025-07-01"

We will visualize NASA remote sensing data from Google Earth Engine: [MOD11A1.061 Terra Land Surface Temperature and Emissivity Daily Global 1km](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD11A1). The land surface temperature data are scaled, so to return to the original temperature in Kelvin, the value needs to be multipled by 0.02. Then, we can convert this to Fahrenheit.

In [ ]:
def to_fahrenheit(lst):
  celsius = lst * 0.02 - 273.15
  fahrenheit = celsius * 1.8 + 32
  return fahrenheit

def to_lst(fahrenheit):
  celsius = (fahrenheit - 32) / 1.8
  lst = (celsius + 273.15) / 0.02
  return lst

In [ ]:
# Load Google Earth Engine data
lst = (
    ee.ImageCollection('MODIS/061/MOD11A1')
      .filterDate(start_date, end_date)
      .select('LST_Day_1km')
      .median()
      .clip(region)
)

# Define color range
colors = [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
]

# Define minimum and maximum values and color scale
lst_vis = {
    'min': to_lst(50),
    'max': to_lst(100),
    'palette': colors,
}

# Add data to map
map.addLayer(lst, lst_vis, "LST")

display(map)

Below is a colorbar that shows what the colors on the map represent: land surface temperature in degrees Fahrenheit.

In [ ]:
plt.figure(figsize=(len(colors), 1))
plt.imshow([ [to_rgb(f'#{c}') for c in colors] ])

plt.text(-0.6, 0.1, '50 °F', va='center', ha='right', fontsize=24)
plt.text(len(colors) - 0.4, 0.1, '100 °F', va='center', ha='left', fontsize=24)

plt.axis('off')
plt.show()

Next, find the average temperature across the county.

In [ ]:
mean_lst = lst.reduceRegion(reducer=ee.Reducer.mean(), geometry=region, scale=1000,
                              maxPixels=1e12).get('LST_Day_1km').getInfo()

print(f"The average temperature for {county_name} from {start_date} to {end_date} is {round(to_fahrenheit(mean_lst), 2)}°F")

Now, we'll visualize the land use and land cover in the county: [USFS Landscape Change Monitoring System v2024.10 (CONUS and OCONUS)](https://developers.google.com/earth-engine/datasets/catalog/USFS_GTAC_LCMS_v2024-10#bands).

In [ ]:
## Land Use

# Reset the map
map = geemap.Map(center=[county_center.y, county_center.x],
                 zoom=10)
map.add_basemap('CartoDB.DarkMatter')

# Define land cover color palette
palette = ['fbff97', 'e6558b', '004e2b', '9dbac5', 'a6976a', '1b1716']
visual = {'min': 1, 'max': 6, 'palette': palette}

# Add 1985 land use
landcover_1985 = (
    ee.ImageCollection('USFS/GTAC/LCMS/v2024-10')
      .filterDate('1985', '1986')
      .filter('study_area == "CONUS"')
      .first()
      .clip(region)
)

map.addLayer(landcover_1985.select('Land_Use'), visual, '1985 Land Use')

# Add 2024 land use
landcover_2024 = (
    ee.ImageCollection('USFS/GTAC/LCMS/v2024-10')
      .filterDate('2024', '2025')
      .filter('study_area == "CONUS"')
      .first()
      .clip(region)
)

map.addLayer(landcover_2024.select('Land_Use'), visual, '2024 Land Use')

display(map)

$$\color{#fbff97}Agriculture$$
$$\color{#e6558b}Developed$$
$$\color{#004e2b}Forest$$
$$\color{#9dbac5}Other$$
$$\color{#a6976a}Rangeland or Pasture$$

We will create a function that calculates the percent of each type of land use and land cover in the county.

In [ ]:
def land_stats(image, name, labels):
  """
  Calculates the percent of each type of land use or land cover in the county.
  Inputs:
  - image: Earth Engine image
  - name: Land_Use or Land_Cover
  - labels: dictionary of labels
  """
  count = image.select(name).reduceRegion(ee.Reducer.frequencyHistogram(), geometry=region, scale=30, maxPixels=1e12).getInfo()

  land_cover_counts = {}

  for key, value in labels.items():
      if key in count[name]:
          land_cover_counts[value] = count[name][key]

  total_pixels = sum([land_cover_counts[i] for i in land_cover_counts])

  for label, num in land_cover_counts.items():
    percent = round(100 * num / total_pixels, 1)
    if percent > 0:
      print(f"{label}: {percent}%")

Get the land use for 1985.

In [ ]:
land_use_labels = {'1': 'Agriculture',
                   '2': 'Developed',
                   '3': 'Forest',
                   '4': 'Other',
                   '5': 'Rangeland or Pasture',
                   '6': 'Non-Processing Area Mask'}

land_stats(landcover_1985, 'Land_Use', land_use_labels)

Now, we repeat for 2024.

In [ ]:
land_stats(landcover_2024, 'Land_Use', land_use_labels)

Visualize land cover on a map.

In [ ]:
## Land Cover

# Reset the map
map = geemap.Map(center=[county_center.y, county_center.x],
                 zoom=10)
map.add_basemap('CartoDB.DarkMatter')

# Define color palette
palette = ['004e2b', '009344', '61bb46', 'acbb67', '8b8560', 'cafd4b', 'f89a1c', '8fa55f', 'bebb8e', 'e5e98a', 'ddb925', '893f54', 'e4f5fd', '00b6f0', '1b1716']
visual = {'min': 1, 'max': 15, 'palette': palette}

# Add land cover for 1985 and 2024
map.addLayer(landcover_1985.select('Land_Cover'), visual, '1985 Land Cover')
map.addLayer(landcover_2024.select('Land_Cover'), visual, '2024 Land Cover')

display(map)

$$\color{#004e2b}Trees$$
$$\color{#61bb46}Shrubs \space and \space Trees Mix$$
$$\color{#acbb67}Grass/Forb/Herb \space and \space Trees Mix$$
$$\color{#8b8560}Barren \space and \space Trees Mix$$
$$\color{#f89a1c}Shrubs$$
$$\color{#8fa55f}Grass/Forb/Herb \space and \space Shrubs Mix$$
$$\color{#bebb8e}Barren \space and \space Shrubs Mix$$
$$\color{#e5e98a}Grass/Forb/Herb$$
$$\color{#ddb925}Barren \space and \space Grass/Forb/Herb Mix$$
$$\color{#893f54}Barren or Impervious$$
$$\color{#e4f5fd}Snow or Ice$$

Find the percent of each type of land cover

In [ ]:
land_cover_labels = {
    '1': 'Trees',
    '2': 'Tall Shrubs & Trees Mix (AK Only)',
    '3': 'Shrubs & Trees Mix',
    '4': 'Grass/Forb/Herb & Trees Mix',
    '5': 'Barren & Trees Mix',
    '6': 'Tall Shrubs (AK Only)',
    '7': 'Shrubs',
    '8': 'Grass/Forb/Herb & Shrubs Mix',
    '9': 'Barren & Shrubs Mix',
    '10': 'Grass/Forb/Herb',
    '11': 'Barren & Grass/Forb/Herb Mix',
    '12': 'Barren or Impervious',
    '13': 'Snow or Ice',
    '14': 'Water',
    '15': 'Non-Processing Area Mask'
}

In [ ]:
land_stats(landcover_1985, 'Land_Cover', land_cover_labels)

In [ ]:
land_stats(landcover_2024, 'Land_Cover', land_cover_labels)

## Tying it all together: Getting Remote Sensing Data for a GLOBE point

View the list of observations for your chosen county.

In [ ]:
data_county

Choose one of these obsarvations and note the index (the left-most number, like 6643 or 29024) of the observation you chose. Write it below.

In [ ]:
index =

# For example,
# index = 29024

Show the data for the point at this index:

In [ ]:
point = data_county[data_county.index == index]
point

Get environmental data (land surface temperature, precipitation, and vegetation) for the point you chose. Read more about this code in [chapter 4 of the digital textbook](https://geo-di-lab.github.io/emerge-lessons/docs/ch4/lesson2.html).

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.
  Args:
      image (ee.Image): A Sentinel-2 image.
  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

def get_data_for_point(point):
    # Extract the point's date property
    point_date = ee.Date(point.get('MeasuredDate'))
    day_before = point_date.advance(-1, 'day')
    month_before = point_date.advance(-30, 'day')

    #### Land Surface Temperature from NASA's MODIS ####
    lst = (
        ee.ImageCollection('MODIS/061/MOD11A1')
        .filterDate(month_before, point_date)
        .select(['LST_Day_1km', 'QC_Day', 'LST_Night_1km', 'QC_Night', 'Clear_day_cov', 'Clear_night_cov'])
        .median()
        # Sample LST at the point’s location
        .reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=point.geometry(),
            scale=1000
        )
    )
    point = point.set(lst)
    point = point.set('LST_Day_1km_F', to_fahrenheit(point.get('LST_Day_1km').getInfo()))

    #### Sentinel-2, Median of Previous Month (used to calculate NDVI) ####
    sentinel2 = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterDate(month_before, point_date)
        .map(mask_s2_clouds)
        .median()
        # Sample median band values at the point’s location
        .reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=point.geometry(),
            scale=10
        )
    )
    point = point.set(sentinel2)

    #### Precipitation from NASA's Daymet V4 ####
    rain = (
        ee.ImageCollection('NASA/ORNL/DAYMET_V4')
        .filterDate(day_before, point_date)
        .select('prcp')     # Daily total precipitation
        .sum()
        # Sample median band values at the point’s location
        .reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=point.geometry(),
            scale=1000
        )
    )
    point = point.set(rain)

    # Add additional information
    b8 = ee.Number(point.get('B8'))
    b4 = ee.Number(point.get('B4'))
    point = point.set('NDVI', b8.subtract(b4).divide(b8.add(b4)))

    return point

Use this function to get precipitation, temperature, and vegetation data for your chosen point.

In [ ]:
point_fc = geemap.geopandas_to_ee(point).first()
result = get_data_for_point(point_fc)

Display the data collected!

The Normalized Difference Vegetation Index (NDVI)  is a value from -1 to 1 representing vegetation levels. High values close to 1 indicate areas of high vegetation like forests, whereas low values close to -1 indicate areas of low vegetation.

In [ ]:
print(f"GLOBE Mosquito observation in {county_name}, {state_name}")
print(f"Date: {result.get('MeasuredDate').getInfo()}")
print(f"Water source: {result.get('WaterSource').getInfo()}")
print(f"Precipitation: {round(result.get('prcp').getInfo(), 2)} mm")
print(f"Land Surface Temperature: {round(result.get('LST_Day_1km_F').getInfo(), 2)} \N{degree sign}F")
print(f"Normalized Difference Vegetation Index (NDVI): {round(result.get('NDVI').getInfo(), 2)}")

Thank you for attending our webinar!

Below are more resources for GLOBE and EMERGE:

- [EMERGE website](https://geoemerge.com/) and [curriculum](https://geo-di-lab.github.io/emerge-lessons)
- [GLOBE Educator Resources](https://www.globe.gov/do-globe/resources/teaching-resources)
- [GLOBE Mosquito Habitat Mapper Resource Library](https://observer.globe.gov/do-globe-observer/mosquito-habitats/resource-library)
- [GLOBE Land Cover Resource Library](https://observer.globe.gov/do-globe-observer/land-cover)